In [1]:
from sqlalchemy import create_engine

In [3]:
import pandas as pd

# 데이터 로드
df = pd.read_csv(r'/Users/eunseo/Downloads/projects/notebooks/sql_db_data.csv')



In [12]:
df

,datetime,vehicle_type,location_name,lane_1,lane_2,lane_3,lane_4,lane_5,lane_6,lane_7,lane_8,lane_9,lane_10,lane_11,lane_12,location_id
0,2023-03-15 07:30:00,버스,공촌3교4거리,1,0,0,1,0,3,0,2,0,6,0,4,1
1,2023-03-15 07:30:00,대형,공촌3교4거리,7,0,6,1,0,0,1,0,1,0,0,6,1
2,2023-03-15 07:30:00,소형,공촌3교4거리,173,1,74,39,12,50,40,22,46,10,3,208,1
3,2023-03-15 07:45:00,버스,공촌3교4거리,1,0,1,0,0,7,0,0,1,3,0,6,1
4,2023-03-15 07:45:00,대형,공촌3교4거리,7,0,2,3,0,0,2,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-11-29 19:00:00,대형,제3청라4거리,1,0,1,0,0,0,0,1,0,0,0,0,10
2876,2023-11-29 19:00:00,소형,제3청라4거리,78,61,172,130,68,26,29,88,0,28,12,23,10
2877,2023-11-29 19:15:00,버스,제3청라4거리,2,1,1,1,0,0,0,3,0,0,0,1,10
2878,2023-11-29 19:15:00,대형,제3청라4거리,0,0,1,1,0,0,0,0,0,0,0,1,10


In [5]:
df.columns

Index(['datetime', 'vehicle_type', 'location_name', 'lane_1', 'lane_2',
       'lane_3', 'lane_4', 'lane_5', 'lane_6', 'lane_7', 'lane_8', 'lane_9',
       'lane_10', 'lane_11', 'lane_12'],
      dtype='object')

In [17]:
df["datetime"] = pd.to_datetime(df["date"] + " " + df["time"])

In [18]:
df = df.drop(columns=['date', 'time','latitude', 'longitude'])

In [13]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [21]:
df = df[[
    "datetime", "vehicle_type", "location_name",
    "lane_1", "lane_2", "lane_3", "lane_4", "lane_5", "lane_6",
    "lane_7", "lane_8", "lane_9", "lane_10", "lane_11", "lane_12"
]]

In [20]:
df.rename(
    columns={
        "vehicle": "vehicle_type",
        "교차로명": "location_name",
        "1": "lane_1",
        "2": "lane_2",
        "3": "lane_3",
        "4": "lane_4",
        "5": "lane_5",
        "6": "lane_6",
        "7": "lane_7",
        "8": "lane_8",
        "9": "lane_9",
        "10": "lane_10",
        "11": "lane_11",
        "12": "lane_12",
    },
    inplace=True,
)


In [9]:
df['location_name'].unique()

array(['공촌3교4거리', '봉수교3거리', '송도1교4거리', '송도2교4거리', '송도3교4거리', '송도4교4거리',
       '송도5교4거리', '제1청라4거리', '제2청라4거리', '제3청라4거리'], dtype=object)

In [22]:
df.to_csv('sql_db_data.csv', index=False)

In [14]:
# MySQL 연결 설정
engine = create_engine("mysql+pymysql://root:비번@localhost/test_db5")

In [15]:
# locations 테이블에서 매핑 정보 가져오기
location_mapping = pd.read_sql("SELECT * FROM locations", engine)
location_dict = dict(zip(location_mapping['name'], location_mapping['id']))


In [16]:
# 데이터프레임의 교차로명을 id로 매핑
df.loc[:, 'location_id'] = df['location_name'].map(location_dict)

# 필요한 컬럼만 선택하여 데이터베이스에 저장
columns_to_save = [
    'location_id','datetime', 'vehicle_type',
    'lane_1', 'lane_2', 'lane_3', 'lane_4', 'lane_5', 'lane_6',
    'lane_7', 'lane_8', 'lane_9', 'lane_10', 'lane_11', 'lane_12',
    
]

# 데이터 업로드
df[columns_to_save].to_sql('traffic_data', engine, if_exists='append', index=False)


2880